In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Paths
root = '/kaggle/input/bird-species-classification-220-categories'
train_path = root + '/Train'
test_path = root + '/Test'

# Get labels
train_labels = [i for i in os.listdir(train_path)]
test_labels = [i for i in os.listdir(test_path)]

tmp_train_img_paths = [train_path + '/' + j for j in train_labels]
tmp_test_img_paths = [test_path + '/' + j for j in test_labels]
train_img_paths, test_img_paths = [], []

for i in tmp_train_img_paths:
    for j in os.listdir(i):
        train_img_paths.append(i + '/' + j)
for i in tmp_test_img_paths:
    for j in os.listdir(i):
        test_img_paths.append(i + '/' + j)
del tmp_test_img_paths, tmp_train_img_paths

random.shuffle(train_img_paths)
random.shuffle(test_img_paths)

# label2name, name2label
label2name, name2label = {}, {}
for i in range(len(train_labels)):
    label2name[i] = train_labels[i]
    name2label[train_labels[i]] = i

# ground truth
train_labels, test_labels = [], []
for i in train_img_paths:
    train_labels.append(name2label[i.split('/')[-2]])
for j in test_img_paths:
    test_labels.append(name2label[j.split('/')[-2]])

# Transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset
class BirdDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item_index):
        img_path = self.data[item_index]
        img_ground_truth = self.labels[item_index]
        img_data = cv2.imread(img_path)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
        if self.transform:
            img_data = self.transform(img_data)
        img_ground_truth_tensor = torch.tensor(img_ground_truth, dtype=torch.long)
        return img_data, img_ground_truth_tensor

# Dataset + Dataloader
train_dataset = BirdDataset(train_img_paths, train_labels, transform=transform)
test_dataset = BirdDataset(test_img_paths, test_labels, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet101(pretrained=True)

for param in model.parameters():
    param.requires_grad = True

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(label2name))
model = model.to(device)

# Loss & optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
epochs = 20
best_accuracy = 0.0

# Training loop
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for batch_data in train_dataloader:
        img_data, label_data = batch_data
        img_data, label_data = img_data.to(device), label_data.to(device)

        optimizer.zero_grad()
        outputs = model(img_data)
        loss = loss_func(outputs, label_data)

        predictions = outputs.argmax(dim=1)
        accuracy = sum(predictions == label_data) / len(label_data)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (predictions == label_data).sum().item()
        total += label_data.size(0)

    epoch_loss = running_loss / len(train_dataloader)
    epoch_accuracy = correct / total
    print(f"Train Loss: {epoch_loss:.6f}, Accuracy: {epoch_accuracy:.6f}")

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct, val_total = 0, 0

    with torch.no_grad():
        for batch_data in test_dataloader:
            img_data, label_data = batch_data
            img_data, label_data = img_data.to(device), label_data.to(device)
            outputs = model(img_data)
            loss = loss_func(outputs, label_data)

            predictions = outputs.argmax(dim=1)
            val_correct += (predictions == label_data).sum().item()
            val_total += label_data.size(0)
            val_loss += loss.item()

    test_loss = val_loss / len(test_dataloader)
    test_accuracy = val_correct / val_total
    print(f"Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}")

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), f'best_model{epoch}.pth')
        print(f"Saved best model! Accuracy: {best_accuracy:.6f}")

print(f'Best accuracy achieved = {best_accuracy}')


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 235MB/s]


Epoch 1/20
Train Loss: 3.728604, Accuracy: 0.236987
Test Loss: 2.529440, Test Accuracy: 0.401432
Saved best model! Accuracy: 0.401432
Epoch 2/20
Train Loss: 1.882452, Accuracy: 0.574251
Test Loss: 1.869770, Test Accuracy: 0.544229
Saved best model! Accuracy: 0.544229
Epoch 3/20
Train Loss: 1.351135, Accuracy: 0.671022
Test Loss: 1.692858, Test Accuracy: 0.560657
Saved best model! Accuracy: 0.560657
Epoch 4/20
Train Loss: 1.076318, Accuracy: 0.746229
Test Loss: 1.540023, Test Accuracy: 0.596883
Saved best model! Accuracy: 0.596883
Epoch 5/20
Train Loss: 0.914268, Accuracy: 0.781071
Test Loss: 1.543483, Test Accuracy: 0.591407
Epoch 6/20
Train Loss: 0.782334, Accuracy: 0.813257
Test Loss: 1.481182, Test Accuracy: 0.592671
Epoch 7/20
Train Loss: 0.669025, Accuracy: 0.838857
Test Loss: 1.497401, Test Accuracy: 0.597725
Saved best model! Accuracy: 0.597725
Epoch 8/20
Train Loss: 0.593383, Accuracy: 0.860314
Test Loss: 1.518669, Test Accuracy: 0.598989
Saved best model! Accuracy: 0.598989
Ep